# Task 1

## Load & Quick Scan


In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
import pandas as pd

file_path = '/content/drive/MyDrive/Copy of consumer_complaints_unclean.csv'
df = pd.read_csv(file_path)

In [148]:
display(df.shape)
display(df.dtypes)

(2040, 16)

Complaint ID                      int64
Date Received                    object
Date Sent to Company             object
Product                          object
Sub-product                      object
Issue                            object
Company                          object
State                            object
ZIP code                         object
City                             object
Company response to consumer     object
Timely response?                 object
Consumer disputed?               object
Latitude                        float64
Longitude                       float64
Status                           object
dtype: object

In [149]:
display(df.head())


Complaint ID Date Received Date Sent to Company        Product  \
0        100000    2023-11-10           2024-01-01       mortgage   
1        100001    2023-03-12           2023-03-21       MORTGAGE   
2        100002    2023-10-20           2023-11-27                  
3        100003    2023-02-17           2023-03-13  Student  loan   
4        100004    2023-11-21           2023-11-23    credit card   

       Sub-product                  Issue        Company State ZIP code  \
0    Interest rate                           Metro Loans    NY    75285   
1      Collections  Collection harassment            NaN    FL    43383   
2    Interest rate                  Other            NaN    NY    43864   
3              NaN                   Fees    Metro Loans    GA      123   
4  Adjustable loan  Collection harassment  United Credit    PA    62086   

         City Company response to consumer Timely response?  \
0   new york                           NaN                N   
1       miami                          NaN              NaN   
2     Unknown                      Unknown            FALSE   
3   new york                           NaN          Unknown   
4    New York                      Unknown              Yes   

  Consumer disputed?   Latitude   Longitude    Status  
0                NaN  42.312932  -91.462816    Closed  
1                  N  42.236514 -103.703588  Resolved  
2                NaN  41.279018  -94.258898    Closed  
3                 No  34.924120  -85.770266            
4                NaN  35.643178  -86.962992   Unknown

#Task 2

## Missing Values

In [150]:
df.isna().sum()

Complaint ID                      0
Date Received                     0
Date Sent to Company              0
Product                          79
Sub-product                     542
Issue                            70
Company                         359
State                           285
ZIP code                          0
City                            292
Company response to consumer    684
Timely response?                248
Consumer disputed?              490
Latitude                          0
Longitude                         0
Status                            0
dtype: int64

In [151]:
#print percentage of null values
missing_percentage = df.isnull().sum() / len(df) * 100
display(missing_percentage)

Complaint ID                     0.000000
Date Received                    0.000000
Date Sent to Company             0.000000
Product                          3.872549
Sub-product                     26.568627
Issue                            3.431373
Company                         17.598039
State                           13.970588
ZIP code                         0.000000
City                            14.313725
Company response to consumer    33.529412
Timely response?                12.156863
Consumer disputed?              24.019608
Latitude                         0.000000
Longitude                        0.000000
Status                           0.000000
dtype: float64

In [152]:
#see all columns with missing values > 80
highly_missing_columns = missing_percentage[missing_percentage > 80]

if highly_missing_columns.empty:
    print("None")
else:
    display(highly_missing_columns)

None


###drop columns with product null and issue null

###Fill sub product, company, state, city, company response to consumer, timely response, consumer disputed



In [153]:
#drop columns with product null and issue null
df_dropped = df.dropna(subset=['Product', 'Issue'])
missing_percentage_after_drop = df_dropped.isnull().sum() / len(df_dropped) * 100
print('Missing percenteges after dropping product and issue with null values')
display(missing_percentage_after_drop)

Missing percenteges after dropping product and issue with null values


Complaint ID                     0.000000
Date Received                    0.000000
Date Sent to Company             0.000000
Product                          0.000000
Sub-product                     26.504752
Issue                            0.000000
Company                         17.370644
State                           14.308342
ZIP code                         0.000000
City                            14.413939
Company response to consumer    33.632524
Timely response?                12.249208
Consumer disputed?              24.709609
Latitude                         0.000000
Longitude                        0.000000
Status                           0.000000
dtype: float64

In [154]:
#Fill sub product, company, state, city, company response to consumer, timely response, consumer disputed
columns_to_fill = ['Sub-product', 'Company', 'State', 'City', 'Company response to consumer', 'Timely response?', 'Consumer disputed?']
df_filled = df_dropped.fillna(dict.fromkeys(columns_to_fill, 'NULL'))
print("dataframe after filling/dropping all null values")
display(df_filled.isnull().sum())

dataframe after filling/dropping all null values


Complaint ID                    0
Date Received                   0
Date Sent to Company            0
Product                         0
Sub-product                     0
Issue                           0
Company                         0
State                           0
ZIP code                        0
City                            0
Company response to consumer    0
Timely response?                0
Consumer disputed?              0
Latitude                        0
Longitude                       0
Status                          0
dtype: int64

#Task 3
##Duplicates

In [155]:
print("Shape before dropping duplicates:", df_filled.shape)

# Count the number of duplicate rows
duplicate_rows = df_filled.duplicated().sum()
print("Number of duplicate rows:", duplicate_rows)

# Drop duplicates by Complaint ID, keeping the latest Date Received
df_no_duplicates = df_filled.sort_values(by='Date Received', ascending=False).drop_duplicates(subset=['Complaint ID'])

print("Shape after dropping duplicates:", df_no_duplicates.shape)

Shape before dropping duplicates: (1894, 16)
Number of duplicate rows: 35
Shape after dropping duplicates: (1761, 16)


#Task 4
##Datatype & Parsing

In [156]:
# Convert date columns to datetime
df_no_duplicates['Date Received'] = pd.to_datetime(df_no_duplicates['Date Received'], errors='coerce')
df_no_duplicates['Date Sent to Company'] = pd.to_datetime(df_no_duplicates['Date Sent to Company'], errors='coerce')

# Convert numeric columns to numeric
df_no_duplicates['ZIP code'] = pd.to_numeric(df_no_duplicates['ZIP code'], errors='coerce')
df_no_duplicates['Latitude'] = pd.to_numeric(df_no_duplicates['Latitude'], errors='coerce')
df_no_duplicates['Longitude'] = pd.to_numeric(df_no_duplicates['Longitude'], errors='coerce')

# Convert categorical/string columns
df_no_duplicates['Product'] = df_no_duplicates['Product'].astype('category')
df_no_duplicates['State'] = df_no_duplicates['State'].astype('category')
df_no_duplicates['City'] = df_no_duplicates['City'].astype('category')

display(df_no_duplicates.dtypes)

Complaint ID                             int64
Date Received                   datetime64[ns]
Date Sent to Company            datetime64[ns]
Product                               category
Sub-product                             object
Issue                                   object
Company                                 object
State                                 category
ZIP code                               float64
City                                  category
Company response to consumer            object
Timely response?                        object
Consumer disputed?                      object
Latitude                               float64
Longitude                              float64
Status                                  object
dtype: object

#Task 5
##Renaming & Category Standardization

In [157]:
# Rename columns to lowercase snake_case
df_no_duplicates.columns = df_no_duplicates.columns.str.replace(' ', '_').str.lower()

# Standardize Product, State, City categories (trim spaces, remove extra spaces, title case)
for col in ['product', 'state', 'city']:
    # Ensure the column is of string type before applying string operations
    df_no_duplicates[col] = df_no_duplicates[col].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True).str.title()


display(df_no_duplicates.head())

complaint_id date_received date_sent_to_company           product  \
580         100580    2024-05-14           2024-06-24  Credit Reporting   
929         100929    2024-05-14           2024-07-01          Mortgage   
1933        101933    2024-05-13           2024-06-09  Credit Reporting   
570         100570    2024-05-13           2024-05-20      Student Loan   
1106        101106    2024-05-13           2024-06-26   Debt Collection   

          sub-product                  issue        company state  zip_code  \
580           Rewards       Billing disputes        Unknown    Tx   71290.0   
929   Adjustable loan  Collection harassment        Unknown    Il   57706.0   
1933  Adjustable loan                   Fees    Metro Loans    Tx   59307.0   
570              NULL  Incorrect information    Metro Loans    Tx   93916.0   
1106  Adjustable loan       Billing disputes  United Credit    Ga  999999.0   

           city company_response_to_consumer timely_response?  \
580     Chicago                      Unknown              Yes   
929    Columbus      Closed with explanation            FALSE   
1933    Unknown                  In progress               No   
570   Charlotte                         NULL             TRUE   
1106    Unknown                         NULL            FALSE   

     consumer_disputed?   latitude  longitude    status  
580               FALSE  39.757075 -78.778719            
929                  No  38.656297 -94.278879  Resolved  
1933                 No  32.338832 -86.062919            
570                NULL  47.654967 -86.313131  Resolved  
1106                 No  33.452865 -91.701748    Closed

Already consistent Product

In [158]:
print(df_no_duplicates['product'].unique())

['Credit Reporting' 'Mortgage' 'Student Loan' 'Debt Collection'
 'Credit Card' 'Bank Account' 'Unknown' '']


#Task 6
##Null-like token, outliers, and sanity check

In [159]:
#check unique values in each column to get an idea of null like token in each column
for col in df_no_duplicates.columns:
    unique_values = df_no_duplicates[col].unique()
    print(f"Unique values in column '{col}':")
    print(unique_values)
    print("\n")

Unique values in column 'complaint_id':
[100580 100929 101933 ... 101242 101169 101248]


Unique values in column 'date_received':
<DatetimeArray>
['2024-05-14 00:00:00', '2024-05-13 00:00:00', '2024-05-12 00:00:00',
 '2024-05-11 00:00:00', '2024-05-10 00:00:00', '2024-05-09 00:00:00',
 '2024-05-08 00:00:00', '2024-05-07 00:00:00', '2024-05-06 00:00:00',
 '2024-05-05 00:00:00',
 ...
 '2023-01-10 00:00:00', '2023-01-09 00:00:00', '2023-01-08 00:00:00',
 '2023-01-07 00:00:00', '2023-01-06 00:00:00', '2023-01-05 00:00:00',
 '2023-01-04 00:00:00', '2023-01-03 00:00:00', '2023-01-02 00:00:00',
 '2023-01-01 00:00:00']
Length: 482, dtype: datetime64[ns]


Unique values in column 'date_sent_to_company':
<DatetimeArray>
['2024-06-24 00:00:00', '2024-07-01 00:00:00', '2024-06-09 00:00:00',
 '2024-05-20 00:00:00', '2024-06-26 00:00:00', '2024-05-17 00:00:00',
 '2024-05-19 00:00:00', '2024-06-27 00:00:00', '2024-06-07 00:00:00',
 '2024-06-16 00:00:00',
 ...
 '2023-02-25 00:00:00', '2023-01-04 00:0

In [160]:
# Replace null-like tokens ('N/A', 'Unknown', '') with NaN
df_no_duplicates.replace(['N/A', 'Unknown', '', 'NULL', ' ','  ','NaN','nan'], pd.NA, inplace=True)


In [161]:
# Standardize 'timely_response?' column: replace TRUE/Yes with Y, and FALSE/No with N
df_no_duplicates['timely_response?'].replace(['TRUE', 'Yes'], 'Y', inplace=True)
df_no_duplicates['timely_response?'].replace(['FALSE', 'No'], 'N', inplace=True)

# Standardize 'consumer_disputed?' column: replace TRUE/Yes with Y, and FALSE/No with N
df_no_duplicates['consumer_disputed?'].replace(['TRUE', 'Yes'], 'Y', inplace=True)
df_no_duplicates['consumer_disputed?'].replace(['FALSE', 'No'], 'N', inplace=True)

print("Unique values in 'timely_response?':")
display(df_no_duplicates['timely_response?'].unique())

print("\nUnique values in 'consumer_disputed?':")
display(df_no_duplicates['consumer_disputed?'].unique())

Unique values in 'timely_response?':


/tmp/ipython-input-1870646817.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_no_duplicates['timely_response?'].replace(['TRUE', 'Yes'], 'Y', inplace=True)
/tmp/ipython-input-1870646817.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].meth

array(['Y', 'N', <NA>], dtype=object)


Unique values in 'consumer_disputed?':


array(['N', <NA>, 'Y'], dtype=object)

In [162]:
# Check for outliers in Latitude/Longitude using IQR
for col in ['latitude', 'longitude']:
    Q1 = df_no_duplicates[col].quantile(0.25)
    Q3 = df_no_duplicates[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df_no_duplicates[(df_no_duplicates[col] < lower_bound) | (df_no_duplicates[col] > upper_bound)]
    print(f"Outliers in {col}:")
    display(outliers)


Outliers in latitude:


complaint_id date_received date_sent_to_company           product  \
107         100107    2024-05-10           2024-05-26       Credit Card   
1460        101460    2024-04-25           2024-06-08   Debt Collection   
1570        101570    2024-04-24           2024-05-28   Debt Collection   
1033        101033    2024-04-10           2024-05-17          Mortgage   
1494        101494    2024-03-08           2024-05-02   Debt Collection   
1858        101858    2024-03-06           2024-03-10  Credit Reporting   
1320        101320    2024-03-03           2024-03-28       Credit Card   
1334        101334    2024-02-11           2024-02-27      Student Loan   
772         100772    2024-02-04           2024-03-02       Credit Card   
1014        101014    2024-02-01           2024-02-20      Bank Account   
1073        101073    2024-01-25           2024-03-20       Credit Card   
138         100138    2024-01-09           2024-01-12          Mortgage   
43          100043    2023-12-02           2023-12-18       Credit Card   
206         100206    2023-11-28           2024-01-20   Debt Collection   
132         100132    2023-11-27           2024-01-14       Credit Card   
941         100941    2023-10-31           2023-11-09  Credit Reporting   
690         100690    2023-10-17           2023-11-23          Mortgage   
1337        101337    2023-10-02           2023-11-07      Bank Account   
366         100366    2023-09-26           2023-11-09   Debt Collection   
416         100416    2023-09-24           2023-10-25       Credit Card   
461         100461    2023-09-18           2023-11-09  Credit Reporting   
734         100734    2023-08-26           2023-10-23      Student Loan   
1479        101479    2023-07-26           2023-09-03       Credit Card   
1675        101675    2023-06-14           2023-07-30   Debt Collection   
508         100508    2023-06-07           2023-07-24          Mortgage   
1076        101076    2023-05-30           2023-07-19      Bank Account   
1677        101677    2023-05-22           2023-05-21          Mortgage   
1416        101416    2023-04-26           2023-05-22          Mortgage   
1749        101749    2023-04-16           2023-05-07  Credit Reporting   
19          100019    2023-03-29           2023-03-24      Bank Account   
463         100463    2023-03-25           2023-04-03      Bank Account   
486         100486    2023-02-28           2023-04-04   Debt Collection   
1196        101196    2023-02-20           2023-03-26  Credit Reporting   
618         100618    2023-01-29           2023-03-15      Bank Account   
1027        101027    2023-01-28           2023-03-12      Bank Account   
423         100423    2023-01-18           2023-01-22       Credit Card   

          sub-product                  issue            company state  \
107        Fixed loan         Identity theft         FinServ Co  Null   
1460    Interest rate  Collection harassment               <NA>  <NA>   
1570  Adjustable loan  Collection harassment        Metro Loans    Oh   
1033       Fixed loan                  Other               <NA>    Oh   
1494       Fixed loan         Identity theft               <NA>    Ga   
1858  Adjustable loan       Billing disputes  Northstar Finance    Ga   
1320             <NA>                   Fees          Acme Bank  Null   
1334             <NA>         Identity theft         FinServ Co    Oh   
772        Fixed loan  Incorrect information      United Credit    Ny   
1014             <NA>                  Other               <NA>    Nc   
1073       Fixed loan  Incorrect information               <NA>    Ca   
138     Interest rate       Billing disputes               <NA>    Nc   
43        Collections       Billing disputes          Acme Bank  Null   
206              <NA>  Incorrect information          Acme Bank    Ga   
132           Rewards  Collection harassment         FinServ Co  Null   
941        Fixed loan                  Other      United Credi

Outliers in longitude:


complaint_id date_received date_sent_to_company           product  \
369         100368    2024-04-28           2024-05-27  Credit Reporting   
208         100208    2024-04-22           2024-05-29       Credit Card   
482         100482    2024-04-10           2024-05-18      Student Loan   
1033        101033    2024-04-10           2024-05-17          Mortgage   
564         100564    2024-04-04           2024-04-03      Student Loan   
1114        101114    2024-04-02           2024-05-14   Debt Collection   
118         100118    2024-03-21           2024-03-30          Mortgage   
1473        101473    2024-03-15           2024-05-01   Debt Collection   
961         100961    2024-03-15           2024-05-08   Debt Collection   
1603        101603    2024-02-13           2024-03-15      Bank Account   
678         100678    2024-01-28           2024-03-24          Mortgage   
7           100007    2024-01-14           2024-02-15          Mortgage   
1859        101859    2024-01-11           2024-02-26      Student Loan   
223         100223    2024-01-10           2024-02-04      Bank Account   
1066        101066    2023-12-02           2023-12-13      Student Loan   
235         100235    2023-11-14           2023-11-15  Credit Reporting   
1918        101918    2023-11-13           2023-11-27      Bank Account   
1787        101787    2023-11-03           2023-11-11   Debt Collection   
2009        100808    2023-10-28           2023-12-11      Bank Account   
948         100948    2023-10-27           2023-11-17              <NA>   
1519        101519    2023-10-13           2023-10-13              <NA>   
1001        101001    2023-09-12           2023-09-20          Mortgage   
1341        101341    2023-05-29           2023-07-20      Bank Account   
456         100456    2023-05-07           2023-05-26      Student Loan   
632         100632    2023-04-11           2023-06-08  Credit Reporting   
644         100644    2023-03-19           2023-03-17       Credit Card   
512         100512    2023-03-15           2023-03-15       Credit Card   
975         100975    2023-02-07           2023-03-10      Student Loan   
147         100147    2023-02-03           2023-03-31          Mortgage   

          sub-product                  issue            company state  \
369           Rewards  Collection harassment  Northstar Finance    Il   
208       Collections       Billing disputes          Acme Bank    Tx   
482   Adjustable loan         Identity theft               <NA>    Il   
1033       Fixed loan                  Other               <NA>    Oh   
564              <NA>  Collection harassment  Northstar Finance    Tx   
1114  Adjustable loan       Billing disputes  Northstar Finance    Oh   
118           Rewards                   Fees        Metro Loans    Pa   
1473  Adjustable loan                   Fees               <NA>  <NA>   
961       Collections  Incorrect information         FinServ Co    Ga   
1603             <NA>                   Fees               <NA>    Tx   
678           Rewards       Billing disputes  Northstar Finance    Ga   
7                <NA>  Incorrect information  Northstar Finance    Tx   
1859          Rewards                   Fees        Metro Loans    Oh   
223     Interest rate  Collection harassment          Acme Bank  <NA>   
1066  Adjustable loan       Billing disputes               <NA>    Mi   
235     Interest rate  Collection harassment      United Credit  <NA>   
1918    Interest rate                   <NA>               <NA>    Ca   
1787       Fixed loan                  Other  Northstar Finance    Pa   
2009  Adjustable loan                  Other               <NA>    Fl   
948           Rewards         Identity theft          Acme Bank  <NA>   
1519             <NA>                  Other         FinServ Co  <NA>   
1001       Fixed loan         Identity theft               <NA>    Nc   
1341          Rewards  Incorrect information               <NA>    Il   
456

In [163]:
# Ensure ZIP codes are valid 5-digit numbers
# Convert to string first to handle potential NaN values after coercion to numeric
df_no_duplicates['zip_code'] = df_no_duplicates['zip_code'].astype(str)
print("\nZIP codes before validation:")
display(df_no_duplicates['zip_code'].head())

# match 5 digits, optionally followed by '.0' for floats converted to string
invalid_zip_codes_mask = ~df_no_duplicates['zip_code'].str.match(r'^\d{5}(\.0)?$', na=False)
invalid_zip_codes = df_no_duplicates[invalid_zip_codes_mask]
print("\nInvalid ZIP codes:")
display(invalid_zip_codes)

# Replace invalid ZIP codes with pd.NA
df_no_duplicates.loc[invalid_zip_codes_mask, 'zip_code'] = pd.NA

print("\nZIP codes after replacing invalid ones with NA:")
display(df_no_duplicates['zip_code'].head())
display(df_no_duplicates['zip_code'].isnull().sum())


ZIP codes before validation:


580      71290.0
929      57706.0
1933     59307.0
570      93916.0
1106    999999.0
Name: zip_code, dtype: object


Invalid ZIP codes:


complaint_id date_received date_sent_to_company           product  \
1106        101106    2024-05-13           2024-06-26   Debt Collection   
1153        101153    2024-05-06           2024-06-21      Student Loan   
1436        101436    2024-05-06           2024-05-29          Mortgage   
958         100958    2024-05-05           2024-06-21  Credit Reporting   
1930        101930    2024-05-04           2024-06-11      Student Loan   
...            ...           ...                  ...               ...   
335         100335    2023-01-04           2023-02-18      Student Loan   
605         100605    2023-01-03           2023-01-27      Student Loan   
1673        101673    2023-01-02           2023-01-09      Student Loan   
1709        101709    2023-01-01           2023-02-28  Credit Reporting   
1242        101242    2023-01-01           2023-01-11       Credit Card   

          sub-product                  issue            company state  \
1106  Adjustable loan       Billing disputes      United Credit    Ga   
1153    Interest rate  Incorrect information  Northstar Finance  Null   
1436          Rewards       Billing disputes        Metro Loans    Nc   
958              <NA>         Identity theft               <NA>    Ca   
1930       Fixed loan                  Other          Acme Bank    Nc   
...               ...                    ...                ...   ...   
335           Rewards                  Other               <NA>    Tx   
605     Interest rate                   Fees               <NA>    Mi   
1673    Interest rate  Incorrect information               <NA>    Fl   
1709      Collections                  Other  Northstar Finance    Il   
1242    Interest rate         Identity theft               <NA>    Nc   

      zip_code         city company_response_to_consumer timely_response?  \
1106  999999.0         <NA>                         <NA>                N   
1153     123.0        Miami                         <NA>                N   
1436     123.0  Los Angeles      Closed with explanation                Y   
958   999999.0     Columbus                         <NA>                N   
1930       0.0      Houston                         <NA>                N   
...        ...          ...                          ...              ...   
335        0.0         Null                       Closed                N   
605        nan      Houston                         <NA>                Y   
1673       nan     Columbus                         <NA>             <NA>   
1709     123.0      Chicago                         <NA>             <NA>   
1242       0.0    Charlotte                       Closed             <NA>   

     consumer_disputed?   latitude   longitude       status  
1106                  N  33.452865  -91.701748       Closed  
1153                  Y  35.510746  -80.706881       Closed  
1436               <NA>  52.059789  -75.648067       Closed  
958                   Y  52.918488 -101.790136         <NA>  
1930               <NA>  52.083899 -111.243264       Closed  
...                 ...        ...         ...          ...  
335                   N  51.543537  -94.721518         <NA>  
605                   Y  36.082684  -86.592140         Open  
1673                  N  43.319832  -88.853647         Open  
1709                  Y  40.121043 -101.005851  In Progress  
1242                  N  50.966459 -119.084642         <NA>  

[370 rows x 16 columns]


ZIP codes after replacing invalid ones with NA:


580     71290.0
929     57706.0
1933    59307.0
570     93916.0
1106       <NA>
Name: zip_code, dtype: object

np.int64(370)

In [164]:
# Create response_time_days
df_no_duplicates['response_time_days'] = (df_no_duplicates['date_sent_to_company'] - df_no_duplicates['date_received']).dt.days

# Select a subset of useful columns
cleaned_df = df_no_duplicates[['complaint_id', 'product', 'issue', 'company', 'state', 'city', 'zip_code', 'status', 'response_time_days']]

# Export to CSV
cleaned_df.to_csv('/content/drive/MyDrive/consumer_complaints_cleaned.csv', index=False)

print("Cleaned data exported to 'consumer_complaints_cleaned.csv'")
display(cleaned_df.head())

Cleaned data exported to 'consumer_complaints_cleaned.csv'


complaint_id           product                  issue        company  \
580         100580  Credit Reporting       Billing disputes           <NA>   
929         100929          Mortgage  Collection harassment           <NA>   
1933        101933  Credit Reporting                   Fees    Metro Loans   
570         100570      Student Loan  Incorrect information    Metro Loans   
1106        101106   Debt Collection       Billing disputes  United Credit   

     state       city zip_code    status  response_time_days  
580     Tx    Chicago  71290.0      <NA>                  41  
929     Il   Columbus  57706.0  Resolved                  48  
1933    Tx       <NA>  59307.0      <NA>                  27  
570     Tx  Charlotte  93916.0  Resolved                   7  
1106    Ga       <NA>     <NA>    Closed                  44